In [3]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam

# Load your data
df = pd.read_csv('Antiasthma.csv')

# Convert 'Sold_date' to datetime format
df['Sold_date'] = pd.to_datetime(df['Sold_date'], format='%m/%d/%y')

# Set 'Sold_date' as the index
df.set_index('Sold_date', inplace=True)

# Aggregate sales data on a weekly basis for each product
weekly_sales = df.groupby([pd.Grouper(freq='W'), 
'Product_details']).agg(total_sold=('Sold_quantity', 'sum')).reset_index()

# Apply one-hot encoding to 'Product_details'
weekly_sales_encoded = pd.concat([weekly_sales, pd.get_dummies(weekly_sales['Product_details'], 
prefix='product')], axis=1).drop('Product_details', axis=1)

# Extract year and week number from 'Sold_date' for temporal features
weekly_sales_encoded['year'] = weekly_sales_encoded['Sold_date'].dt.year
weekly_sales_encoded['week_of_year'] = weekly_sales_encoded['Sold_date'].dt.isocalendar().week

# Normalize the temporal features 
scaler = MinMaxScaler()
features_columns = ['year', 'week_of_year'] + [col for col in weekly_sales_encoded.columns 
if col.startswith('product')]
weekly_sales_encoded[features_columns] = scaler.fit_transform(weekly_sales_encoded[features_columns])

# Split the data into features and target
X = weekly_sales_encoded[features_columns].values
y = weekly_sales_encoded['total_sold'].values

# Adjusting the dataset split into 70% training, 15% validation, and 15% testing
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1765, random_state=42)  # 0.1765 of 85% is roughly 15% of the whole

# Define the model architecture
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(100, activation='relu'),  # Additional hidden layer
    Dense(100, activation='relu'),  # Additional hidden layer
    Dense(100, activation='relu'),  # Additional hidden layer
    Dense(1, activation='relu')    # Output layer
])


# Compile the model
model.compile(optimizer=Adam(), loss='mean_squared_error')

# Train the model with the validation data
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=10)

# Evaluate the model
predictions = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print(f'RMSE on test set: {rmse}')

# Prepare forecasting features for Feb 1 to 8, 2024, correctly this time
forecast_results = []
for product_col in [col for col in features_columns if col.startswith('product')]:
    forecast_period_features = np.zeros((1, len(features_columns)))
    # Find the index of the year, week_of_year, and current product column
    year_index = features_columns.index('year')
    week_of_year_index = features_columns.index('week_of_year')
    product_index = features_columns.index(product_col)
    
    # Normalize year and week_of_year values
    forecast_period_features[0, year_index] = (2024 - scaler.data_min_[year_index]) / (scaler.data_max_[year_index] - scaler.data_min_[year_index])
    forecast_period_features[0, week_of_year_index] = (5 - scaler.data_min_[week_of_year_index]) / (scaler.data_max_[week_of_year_index] - scaler.data_min_[week_of_year_index])
    forecast_period_features[0, product_index] = 1  # Activate current product

    # Predict and store the result
    predicted_quantity = model.predict(forecast_period_features).flatten()[0]
    product_name = product_col.replace('product_', '')  # Assuming prefix 'product_' is used
    forecast_results.append((product_name, predicted_quantity))

# Display forecasted quantities for each product in a DataFrame
forecast_df = pd.DataFrame(forecast_results, columns=['Product', 'Forecasted Quantity']).sort_values(by='Forecasted Quantity', ascending=False)
# Save forecast_df to an Excel file
forecast_df.to_excel('antiasthma_results.xlsx', index=False)

Epoch 1/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 642us/step - loss: 318.7273 - val_loss: 166.8529
Epoch 2/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 374us/step - loss: 171.3460 - val_loss: 154.3451
Epoch 3/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 376us/step - loss: 120.4993 - val_loss: 150.0947
Epoch 4/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 367us/step - loss: 152.0093 - val_loss: 150.8098
Epoch 5/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 369us/step - loss: 132.3763 - val_loss: 152.8472
Epoch 6/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 371us/step - loss: 110.9060 - val_loss: 151.0727
Epoch 7/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 370us/step - loss: 121.0068 - val_loss: 160.3180
Epoch 8/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 367us/step - loss: 154.2642 - val_loss: 156.5378
Epoch 9/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 368us/step - loss: 131.9047 - val_loss: 162.2818
Epoch 10/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 367us/step - loss: 132.9830 - val_loss: 154.8610
Epoch 11/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 367us/step - loss: 153.6

143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 382us/step - loss: 117.3231 - val_loss: 151.2881
Epoch 66/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 374us/step - loss: 124.2043 - val_loss: 156.5866
Epoch 67/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 372us/step - loss: 110.5137 - val_loss: 153.7854
Epoch 68/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 387us/step - loss: 98.2428 - val_loss: 152.5848
Epoch 69/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 367us/step - loss: 121.1403 - val_loss: 150.4484
Epoch 70/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 366us/step - loss: 130.1042 - val_loss: 153.4557
Epoch 71/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 367us/step - loss: 118.3640 - val_loss: 155.6815
Epoch 72/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 367us/step - loss: 118.7121 - val_loss: 151.7376
Epoch 73/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 368us/step - loss: 109.7717 - val_loss: 152.1440
Epoch 74/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 367us/step - loss: 105.6022 - val_loss: 160.2967
Epoch 75/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 367us/step - loss: 107.7326 -